In [4]:
import numpy as np
from scipy.optimize import linprog

def generate_bids(n, m, p_bar):
    """
    Generate a sequence of random bids.
    :param n: Total number of bids.
    :param m: Number of items.
    :param p_bar: Ground truth price vector.
    :return: Array of bids.
    """
    bids = []
    for _ in range(n):
        a_k = np.random.choice([0, 1], size=m)  # Generate a_k
        pi_k = np.dot(p_bar, a_k) + np.random.normal(0, np.sqrt(0.2))  # Calculate bid price
        bids.append((a_k, pi_k))
    return bids

def solve_offline_lp(bids, m, b_i):
    c = -np.array([pi_k for _, pi_k in bids])  # Negative for maximization
    A = np.array([a_k for a_k, _ in bids]).T  # Transpose to match dimensions
    b = b_i * np.ones(m)

    # Solving the LP
    result = linprog(c, A_ub=A, b_ub=b, bounds=(0, 1), method='highs')

    if result.success:
        return -result.fun  # Revenue (negate because of maximization)
    else:
        raise ValueError("Offline LP did not converge")

def solve_partial_lp_dual(bids, k, n, b_i):
    # Objective function: maximize sum(pi_j * x_j) for j=1 to k
    c = -np.array([pi_k for _, pi_k in bids[:k]])

    # Constraints: sum(a_ij * x_j) <= (k/n) * b_i for all i
    A = np.array([a_k for a_k, _ in bids[:k]]).T
    b = (k / n) * np.array(b_i)

    # Bounds for decision variables: 0 <= x_j <= 1
    x_bounds = [(0, 1) for _ in range(k)]

    # Solve the linear program
    result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

    # print(result.ineqlin.get('marginals'))

    if result.success:
        # The dual variable corresponding to the inequality constraints A_ub * x <= b_ub
        return result.get('slack'), -1*result.ineqlin.get('marginals'), result.get('fun')
    else:
        raise ValueError("failed to find a solution")

In [5]:
offline_revenue_value = 0
diff_dplm = []
diff_ahdla = []
revenues = {}

n = 10000  # Total number of bids
m = 10     # Number of items
b_i = np.ones(m) * 1000  # Bid cap for all i
# Fixed ground truth price vector (p_bar) - set to ones for simplicity
p_bar = np.ones(m)  # Vector of ones
k_values = [50, 100, 200]  # Different k values to test
# Regenerate bids with the fixed p_bar
bids_fixed = generate_bids(n, m, p_bar)

# offline
offline_revenue_value = solve_offline_lp(bids_fixed, m, b_i)
print(f"Offline revenue: {offline_revenue_value}")

Offline revenue: 11291.587193125308


In [6]:
# Problem 1
def run_slpm_static(bids, k, n, b_i):
    revenue = 0
    remaining_capacity = np.array(b_i)
    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    # revenue += -k_revenue
    # remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k

    return revenue

for k in k_values:
    revenue = run_slpm_static(bids_fixed, k, n, b_i)
    revenues[k] = revenue

for slpm_revenue, k in zip(revenues.values(), k_values):
    print(f"SLPM Static revenue: {slpm_revenue} at k={k}")

SLPM Static revenue: 9587.554808858562 at k=50
SLPM Static revenue: 9588.103476894199 at k=100
SLPM Static revenue: 10323.377974335992 at k=200


The increase in revenue as k grows suggests that having more bidder information upfront (i.e., a larger k) allows the algorithm to make more informed decisions which result in higher revenue.

The substantial increase in revenue from k=100 to k=200 indicates a possible threshold effect, where having a certain amount of initial information (in this case, the bids of 200 bidders) can significantly improve the algorithm's performance.

It may have two tradeoffs here, one is tradeoff between cost of computation and accuracy, since with a large k we will need more compuration resources. Another tradeoff is that a larger k implies a delay in decision-making since more bids must be collected before making allocations, which could be a trade-off in real-time scenarios.


In [7]:
# Problem 2
def run_slpm_dynamic(bids, k, n, b_i):
    global offline_revenue_value
    global diff_dplm
    revenue = 0
    remaining_capacity = np.array(b_i)

    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    # revenue += -k_revenue
    # remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    target = 2*k

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            if i == target:
                slack, dual_price, _ = solve_partial_lp_dual(bids, target, n, b_i)
                print(dual_price)
                target *= 2
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k
            diff_dplm.append(revenue-((i+1)/n)*offline_revenue_value)


    return revenue

revenue = run_slpm_dynamic(bids_fixed, 50, n, b_i)
print(f"SLPM Dynamic revenue: {revenue}")
print(diff_dplm)

[1.00123977 1.12393472 1.12282009 0.90103336 1.36074724 1.05777959
 1.0140114  0.82332313 1.04998882 1.16299426]
[1.0881418  1.02543201 1.02987887 0.99457621 1.27880507 1.17721059
 1.05534733 0.88627075 1.05521663 0.99326804]
[1.2283813  1.10709726 1.04555257 1.05749631 1.14886413 0.97011559
 1.00185853 0.86260548 0.97748679 1.19585186]
[1.10386615 1.12228368 1.05816994 1.08611959 1.08207524 1.012015
 1.03684379 1.00337407 0.99920802 1.16957266]
[1.08173249 1.10920479 1.05471092 1.04999453 1.08476734 1.04121986
 1.06847466 1.0506862  1.0008501  1.15371288]
[1.067286   1.08798334 1.0502676  1.06840125 1.09889093 1.0577844
 1.0633338  1.08160667 1.03010764 1.09534975]
[1.06950816 1.06882622 1.0553033  1.07357239 1.07260014 1.09314754
 1.06228981 1.0574869  1.0605227  1.07017259]
SLPM Dynamic revenue: 10905.843288804756
[-57.587094684939075, -58.716253404251596, -53.07161621727496, -54.20077493658749, -50.429928890162614, -51.55908760947514, -45.0930884180176, -46.22224713733013, -47.3514

In this question, instead of using a fixed dual price, we update our dual price on k= [50, 100, 200, 400, 800, ...], and using this stratege, we can get better performance than with fixed dual price, 10905.843288804756 compare to revenue in question above.

Also we print all dual price on each update time point. As we can see, it approches to the ground true price p = [1,1,...,1,1] we set before.

In [8]:
# Problem 3
def run_ahdla(bids, k, n, b_i):
    global offline_revenue_value
    global diff_ahdla
    batch = 50
    revenue = 0
    dual_price = np.zeros(m)
    remaining_capacity = b_i
    target = batch*k

    for i, (a_k, pi_k) in enumerate(bids):
        # Allocate based on the decision rule using y_bar
        if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
            revenue += pi_k
            remaining_capacity -= a_k
        diff_ahdla.append(revenue-((i+1)/n)*offline_revenue_value)
        if i == target and i < n-1:
            _, dual_price, _ = solve_partial_lp_dual(bids, i+1, n, (n/(n-i-1))*remaining_capacity)
            target += batch

    return revenue

revenue = run_ahdla(bids_fixed, 1, n, b_i)
print(f"Action-history-dependent Learning Algorithm revenue: {revenue}")
print(diff_ahdla)

Action-history-dependent Learning Algorithm revenue: 11237.926050273081
[3.9018285412095945, 6.5928190332285155, 9.301987005948869, 14.08521462392601, 19.72062867793245, 22.487306079696815, 25.83509858783874, 30.718152182719628, 33.46342247794087, 39.13849872998651, 40.46149345819558, 44.3274539080755, 51.04344100664208, 54.07634077766243, 56.65429813896547, 61.45513534587448, 66.77206050666207, 70.59610249377256, 71.80242233640371, 76.78284853817664, 80.96763663645476, 85.03191211751015, 89.0822202804913, 95.14615907999122, 100.44938606948288, 105.58032136707938, 111.20488859718803, 115.80185926048308, 117.37414900079565, 120.29793603207028, 123.85634992298253, 127.51172649538825, 132.65449181075215, 139.89887770291816, 141.00783547882966, 145.92135331647955, 154.87615302099726, 158.40868191649878, 161.32855940458364, 163.3800604813847, 165.9124878690608, 170.3193259810394, 172.33795543032568, 175.04243365747976, 176.24868713077183, 181.94777504075861, 183.36048907519069, 184.76722222

In Action-history-dependent Learning Algorithm, instead of updating dual price in a frequency of [50, 100, 200, ...], we update it for each batch, we set 50 here. And as we can see, the revenue is higher than what we got from problem 2.

Hence Action-history-dependent Learning Algorithm have better performance. Becuase it also update with remaining capacity information.

As we can see in the difference vector, compared with problem 2, (see result on problem2), Action-history-dependent Learning Algorithm is closer to partial of optimal solution at each time point.

# Problem 4
## Convexity

The Formulation (3) in question is as follows:

\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}

where $\mathbf{d} = \frac{\mathbf{b}}{n}$ and $(\cdot)^+ = \max \{\cdot, 0\}$.

To identify whether (3) is a convex optimization problem or not, we can analyze each part of (3) separately.

For the first part $\mathbf{d}^T \bar{\mathbf{y}}$ of (3), $\mathbf{d}^T \bar{\mathbf{y}}$ is liner in $\bar{\mathbf{y}}$. We know that linear functions are both convex and concave.

For the second part $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$, since $\pi - \mathbf{a}^T \bar{\mathbf{y}}$ is linear in $\bar{\mathbf{y}}$. What $(\cdot)^+$ does is to get maximum value between this linear function and zero, since both of them are convex, $\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is also convex. The last part we need to consider is the expectation. From Jensen's inequality:

$$\mathbb{E}[f(X)]≥f(\mathbb{E}[X])$$

we know that if $f(x)$ is convex, then $\mathbb{E}[f(x)]$ is also convex. Therefore, $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is convex.

Since both parts of (3) are convex, we know that (3) is a convex optimization problem.

## Connection

The dual problem of (1) is:

$$
\min \sum_{i=1}^{m} b_iy_i + \sum_{j=1}^{n} \beta_j\\
\text{s.t.} \sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j, \quad j=1,\ldots,n.\\
y_i, \beta_j \geq 0 \text{ for all } i, j
$$

From previous questions, we know that the primal optimal solution satisfies:
$$
x_j^* =
\begin{cases}
1, & \text{if } \pi_j > a_j^T y_k^* \\
0, & \text{if } \pi_j \leq a_j^T y_k^*.
\end{cases}
$$

the optimal solution $x_j$ may take non-integer values. That means the optimal solution of primary problem (1) highly depends on $y$. So by plugging the constraints $\sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j$ into the objective function, an equivalent form of the dual problem can be obtained as:
$$
\min \sum_{i=1}^{m} b_i y_i + \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

And if we devide it by $n$, from above we know that $\mathbf{d} = \frac{\mathbf{b}}{n}$, so we now have:
$$
\min f_n(\bar{\mathbf{y}}) = \sum_{i=1}^{m} d_i y_i + \frac{1}{n} \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

Since $(\pi, a)$ is a sequence of i.i.d. random vectors, we can also write this formulation as:

$$
\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}
$$

where the expectation is taken with respect to $(\pi, a)$.
